**Instruções de uso:**

Crie uma máquina virtual: python -m venv .venv  
Ative a maquina virtual: .venv\Scripts\activate  
Instale as dependências: pip install -r requirements.txt  

In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

**Solicitação Nº1**  
Será necessário unir os arquivos disponíveis na pasta “Vendas” de maneira dinâmica, de uma forma que ao
acrescentar mais um arquivo será necessário apenas rodar o código novamente e não escrever outra linha de
código.

In [30]:
pasta = '../Vendas/'

In [ ]:
arquivos = [
    os.path.join(pasta, arquivo)
    for arquivo in os.listdir(pasta)
    if arquivo.startswith('Vendas') and arquivo.endswith('.csv') # Garante que só pegue arquivos csv que começam com 'Vendas'
]

In [ ]:
dfs = [pd.read_csv(arquivo) for arquivo in arquivos] # Une todos os dataframes em uma lista

In [ ]:
df_vendas = pd.concat(dfs, ignore_index=True) # Concatena todos os dataframes em um único dataframe

**Análise e ajuste inicial dos atributos**

In [35]:
df_vendas.head()

,cdCliente,cdProduto,cdVendedor,valor,dtVenda
0,138,1940,9,820,18/02/2020
1,154,1551,7,815,03/02/2020
2,148,1844,1,225,18/01/2020
3,183,1827,4,915,22/01/2020
4,143,1959,5,169,27/03/2020


In [37]:
df_vendas.shape

(500, 5)

In [38]:
df_vendas.info()

<class 'pandas.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   cdCliente   500 non-null    int64
 1   cdProduto   500 non-null    int64
 2   cdVendedor  500 non-null    int64
 3   valor       500 non-null    int64
 4   dtVenda     500 non-null    str  
dtypes: int64(4), str(1)
memory usage: 19.7 KB


In [ ]:
df_vendas['dtVenda'] = df_vendas['dtVenda'].astype('datetime64[ns]') # Converte a coluna dtVenda para o tipo datetime

In [40]:
df_vendas.info()

<class 'pandas.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   cdCliente   500 non-null    int64         
 1   cdProduto   500 non-null    int64         
 2   cdVendedor  500 non-null    int64         
 3   valor       500 non-null    int64         
 4   dtVenda     500 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(4)
memory usage: 19.7 KB


**Solicitação Nº2**  
Após unir os arquivos foi informado que o Vendedor 10 se trata de um erro no sistema, então o mesmo deverá
ser excluído do dataframe.

Ps: Ao analisar a base de dados de vendedores, nota-se que o vendedor de id 10 realmente não existe.

- Excluir esses dados poderá interferir no faturamento total das vendas.

In [41]:
df_vendas_vend_10 = df_vendas[df_vendas['cdVendedor'] == 10].reset_index(drop=True)

In [ ]:
df_vendas_vend_10.shape # Verifica o número de linhas e colunas do dataframe filtrado

In [42]:
df_vendas = df_vendas[df_vendas['cdVendedor'] != 10].reset_index(drop=True)

In [61]:
df_vendas.shape # Verifica o número de linhas e colunas do dataframe após a remoção do vendedor 10

(435, 6)

In [ ]:
df_vendas[df_vendas['cdVendedor'] == 10] # Verifica se ainda existem registros do vendedor 10 no dataframe

,id_venda,dtVenda,cdVendedor,cdProduto,cdCliente,valor


**Solicitação Nº3**  
A empresa está fazendo 7 anos de existência, e todas as vendas efetuadas pelo vendedor 7 devem ser precificadas com 70 reais no dataframe.

Ps: Essa solicitação realmente faz sentido?  
- Atribuir o valor fixo de R$ 70,00 a todas as vendas do vendedor 7, com base apenas no tempo de existência da empresa, não parece refletir uma regra de negócio comum.

- Considerar essa condição apenas como um exercício técnico ou revisar a lógica de precificação.

In [43]:
# df_vendas[df_vendas['cdVendedor'] == 7]['valor'] = 70.0

**Solicitação Nº4**  
O arquivo deverá ser exportado para a pasta “Destino Vendas”, por fim para um estudo isolado da equipe de
vendas, será necessário a exportação de um outro dataframe com as informações de vendas apenas do
vendedor 10.

In [ ]:
df_vendas_vend_10.to_csv('../Destino Vendas/vendas_vendedor_10.csv', index=False) # Salva as vendas do vendedor 10 em um novo arquivo CSV

In [63]:
if os.path.exists('../Destino Vendas/vendas_vendedor_10.csv'):
    print("Arquivo vendas_vendedor_10.csv criado com sucesso!")
else:
    print("Erro ao criar o arquivo vendas_vendedor_10.csv.")

Arquivo vendas_vendedor_10.csv criado com sucesso!


In [ ]:
df_vendas['id_venda'] = df_vendas.index + 1 # Cria uma coluna id_venda como chave primária

In [ ]:
df_vendas = df_vendas[['id_venda', 'dtVenda', 'cdVendedor', 'cdProduto', 'cdCliente', 'valor']] # Reordena as colunas

In [ ]:
df_vendas.to_csv('../Destino Vendas/Total_Vendas.csv') # Salva o dataframe final em um arquivo CSV

In [ ]:
if os.path.exists('../Destino Vendas/Total_Vendas.csv'):
    print("Arquivo Total_Vendas.csv criado com sucesso!")
else:
    print("Erro ao criar o arquivo Total_Vendas.csv.")

Arquivo Total_Vendas.csv criado com sucesso!


**Lendo e ajustando os demais arquivos**

In [49]:
df_clientes = pd.read_csv('../Clientes.csv', sep=';')

In [ ]:
df_clientes.rename(columns={'Cd Cliente': 'cdCliente'}, inplace=True) # Renomeia a coluna para combinar com o dataframe de vendas

In [51]:
df_produtos = pd.read_csv('../Produto.txt', sep='|', encoding='latin1')

In [ ]:
df_produtos.rename(columns={'Desc Produto': 'descProduto', 'Quantidade em estoque': 'qtdEstoque'}, inplace=True) # Renomeia as colunas para combinar com o dataframe de vendas

In [53]:
df_vendedores = pd.read_csv('../Vendedor.txt', sep='\t')

In [ ]:
df_vendedores.rename(columns={'Nome Vendedor': 'nome'}, inplace=True) # Renomeia a coluna para combinar com o dataframe de vendas

**Conexão com o BD**

In [ ]:
load_dotenv()  # Carrega variáveis do .env

endpoint = os.getenv("DB_ENDPOINT")
database = os.getenv("DB_NAME")
username = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")

connection_string = (
    f"mssql+pyodbc://{username}:{password}@{endpoint}:1433/{database}"
    "?driver=ODBC+Driver+17+for+SQL+Server"
    "&TrustServerCertificate=yes"
) # String de conexão com o banco de dados SQL Server

engine = create_engine(connection_string, fast_executemany=True) # Cria a engine de conexão com o banco de dados

In [ ]:
df_vendas.to_sql(
    "fato_vendas",
    con=engine,
    schema="dbo",
    if_exists="append",
    index=False,
) # Carrega o dataframe de vendas para a tabela fato_vendas no banco de dados

-1

In [ ]:
df_clientes.to_sql(
    "clientes",
    con=engine,
    schema="dbo",
    if_exists="append",
    index=False,
) # Carrega o dataframe de clientes para a tabela clientes no banco de dados

-1

In [ ]:
df_produtos.to_sql(
    "produtos",
    con=engine,
    schema="dbo",
    if_exists="append",
    index=False,
) # Carrega o dataframe de produtos para a tabela produtos no banco de dados

-1

In [ ]:
df_vendedores.to_sql(
    "vendedores",
    con=engine,
    schema="dbo",
    if_exists="append",
    index=False,
) # Carrega o dataframe de vendedores para a tabela vendedores no banco de dados

-1